In [12]:
from transformers import pipeline
import torch


In [13]:
pipe = pipeline("text-classification")
pipe("This restaurant is awesome")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998743534088135}]

In [14]:
class Spy(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *args, **kwargs):
        print(f'args {args}')
        print(f'kwargs {kwargs}')
        output = self.model(*args, **kwargs)
        print(f'output {output}')
        print(output[0].shape)
        return output

In [15]:
pipe.model.distilbert.transformer.layer

ModuleList(
  (0-5): 6 x TransformerBlock(
    (attention): MultiHeadSelfAttention(
      (dropout): Dropout(p=0.1, inplace=False)
      (q_lin): Linear(in_features=768, out_features=768, bias=True)
      (k_lin): Linear(in_features=768, out_features=768, bias=True)
      (v_lin): Linear(in_features=768, out_features=768, bias=True)
      (out_lin): Linear(in_features=768, out_features=768, bias=True)
    )
    (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (ffn): FFN(
      (dropout): Dropout(p=0.1, inplace=False)
      (lin1): Linear(in_features=768, out_features=3072, bias=True)
      (lin2): Linear(in_features=3072, out_features=768, bias=True)
      (activation): GELUActivation()
    )
    (output_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  )
)

In [16]:
pipe.model.distilbert.transformer.layer[0] = Spy(pipe.model.distilbert.transformer.layer[0])

In [17]:
pipe("This restaurant is awesome and I like it a lot, but I don't like the food.")

args (tensor([[[ 0.3549, -0.1386, -0.2253,  ...,  0.1536,  0.0748,  0.1310],
         [-0.7462,  0.7479,  0.1150,  ...,  0.3907,  0.3402,  0.3951],
         [ 0.6612, -0.1107, -0.6210,  ..., -0.6172,  1.0782, -0.0198],
         ...,
         [-0.1384,  0.4437,  0.0756,  ...,  0.5573,  0.6575, -0.8687],
         [ 0.0614,  0.1503, -0.3501,  ...,  0.4288,  0.3237,  0.1198],
         [-0.2599,  0.0690, -0.3147,  ..., -0.3003,  0.2748, -0.1766]]]), tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), None, False)
kwargs {}
output (tensor([[[ 0.0215,  0.0527, -0.1512,  ...,  0.0568,  0.0205,  0.0351],
         [-0.3558,  0.7178,  0.2859,  ...,  0.1407,  0.1163,  0.2365],
         [ 0.8229,  0.1483, -0.4060,  ..., -0.3931,  1.0438, -0.4586],
         ...,
         [ 0.1851,  1.0592, -0.0766,  ...,  0.2654,  0.3442, -1.0609],
         [ 0.1812,  0.2816, -0.0501,  ...,  0.1309, -0.0092, -0.3293],
         [-0.2396, -0.0301, -0.1492,  ...,  0.0310,  0.0384, -0.1757]]]),

[{'label': 'NEGATIVE', 'score': 0.9341132640838623}]